In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
data = pd.read_csv('../input/spaceship-titanic/train.csv')
data.head()

In [ ]:
test = pd.read_csv('../input/spaceship-titanic/test.csv')
submission = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')

In [ ]:
data.isnull().sum()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
sns.countplot(x = 'HomePlanet',  data = data )
plt.show()

In [ ]:
data.Cabin.nunique()

In [ ]:
sns.heatmap(data.isnull() , cbar = False)
plt.show()

In [ ]:
# import itertools
# data1 = pd.DataFrame({'cnt': list(itertools.chain(*[[row['index']] * row['cnt'] for idx, row in data.isnull().sum().to_frame('cnt').reset_index().iterrows()]))})
# plt.figure(figsize = (20,10))
# sns.countplot(x = 'cnt', data = data1)
# plt.show()
import missingno as msno
msno.bar(data)
plt.show()

In [ ]:
data.dtypes

In [ ]:
other_than_object = data.dtypes[data.dtypes != 'object']
other_than_object.index

In [ ]:
corr= data.corr()
sns.heatmap(corr , cmap = 'coolwarm' , annot = True , vmin = -1 , vmax = 1)
plt.show()

> # ****Seeing this we can say data doesnot have multicollinearity****

In [ ]:
y = data.Transported
data.drop(['Transported']  ,axis =1 , inplace = True)

In [ ]:
mix_data = pd.concat([data , test] , ignore_index= True)
mix_data.shape

In [ ]:
#we can safely say that passenger id and name cannot be deciding factor so drop those
mix_data.drop(['PassengerId' , 'Name'] , inplace = True , axis = 1)

In [ ]:
only_object = mix_data.dtypes[mix_data.dtypes == 'object']
only_object.index
for i in only_object.index:
    print(i , end = ' ')
    print(mix_data[i].nunique())

In [ ]:
mix_data.Cabin.isnull().sum()

In [ ]:
mix_data.isnull().sum()

In [ ]:
#filling the object value with mode
for i in only_object.index:
    print(i , mix_data[i].mode()[0])
    mix_data[i] = mix_data[i].fillna(mix_data[i].mode()[0])

In [ ]:
other_than_object = data.dtypes[data.dtypes != 'object']
other_than_object.index

In [ ]:
mix_data.head()

In [ ]:
mix_data[mix_data.index == 0]

In [ ]:
%%time
#filling the integer value with mean technique
from sklearn.impute import SimpleImputer
for i in other_than_object.index:
    imp = SimpleImputer(strategy='median')
    mix_data[i] = imp.fit_transform(mix_data[i].values.reshape(-1,1))

In [ ]:
mix_data.isnull().sum()

# Our dataset is free of missing value

In [ ]:
#cabin can be important feature but let's first try without it because it has lots of unique categorical value
mix_data.drop(['Cabin'] , axis = 1 , inplace= True)

In [ ]:
only_object = mix_data.dtypes[mix_data.dtypes == 'object']
only_object.index

In [ ]:
mix_data.dtypes

In [ ]:
cat_val = ['HomePlanet' , 'Destination' , 'CryoSleep' , 'VIP']
from sklearn.preprocessing import LabelEncoder
for i in cat_val:
    le = LabelEncoder()
    mix_data[i] = le.fit_transform(mix_data[i])

In [ ]:
mix_data.dtypes

In [ ]:
#now our data is categoorical value free , now we have to normalize our data
mix_data.describe().T.sort_values(by='std' , ascending=False)\
                     .style.background_gradient(cmap='GnBu')\
                     .bar(subset=["max"], color='#BB0000')\
                     .bar(subset=["mean",], color='green')

In [ ]:
train = mix_data.iloc[:len(data) , :]
test = mix_data.iloc[len(data): , :]

In [ ]:
train.shape , test.shape

In [ ]:
y.unique()

In [ ]:
%%time
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from tqdm import tqdm
lgb_params = {
    'objective' : 'binary',
    'n_estimators' :50,
    'learning_rate' : 0.08
}
lgb_predictions = 0
lgbm = LGBMClassifier(**lgb_params)
skf = StratifiedKFold(n_splits = 5 , shuffle = True)
for fold , (train_idx , valid_idx) in tqdm(enumerate(skf.split(train , y))):
    print(10*"=", f"Fold={fold+1}", 10*"=")
    X_train, X_valid = train.iloc[train_idx] , train.iloc[valid_idx]
    y_train , y_valid = y.iloc[train_idx] , y.iloc[valid_idx]
    model = lgbm.fit(X_train , y_train)
    preds_valid = model.predict(X_valid)
    acc = accuracy_score(y_valid,  preds_valid)
    print(f"Fold={fold+1}, Accuracy score: {acc:.2f}%")

In [ ]:
submission.head()

In [ ]:
prediction = lgbm.predict(test)

In [ ]:
submission['Transported'] = prediction
submission.to_csv("submission.csv",index=False)
submission.head()